# Notebook 01: Pipeline d'Ingestion et d'Embedding des Données ArXiv

Ce notebook démontre le processus complet d'acquisition, de traitement, d'embedding et de stockage des articles scientifiques d'ArXiv dans notre base de données MongoDB.

**Prérequis :**
* Assurez-vous d'avoir exécuté le notebook `00_setup_environment.ipynb` et que votre environnement est correctement configuré (variables d'environnement chargées, clés API valides, MongoDB accessible).
* Les bibliothèques nécessaires doivent être installées via `environment.yml`.

**Étapes de ce Notebook :**
1.  Configuration initiale (imports, logging, connexion MongoDB).
2.  Téléchargement d'articles depuis ArXiv.
3.  Parsing des documents PDF et de leurs métadonnées.
4.  Prétraitement du texte (nettoyage et chunking).
5.  Génération des embeddings pour les chunks.
6.  Stockage des chunks et de leurs embeddings dans MongoDB et création des index.

In [ ]:
import logging
import sys
from pathlib import Path
import os
from typing import List

# S'assurer que la racine du projet est dans le PYTHONPATH pour les imports de src
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"Ajout de {project_root} au PYTHONPATH")

# Charger les variables d'environnement (si .env est à la racine du projet)
from dotenv import load_dotenv
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il existe et est configuré.")


# Importer nos modules
from config.settings import settings
from config.logging_config import setup_logging
from src.data_processing.arxiv_downloader import download_pipeline as download_arxiv_papers
from src.data_processing.document_parser import parse_document_collection, PDF_INPUT_DIR, METADATA_INPUT_DIR
from src.data_processing.preprocessor import preprocess_parsed_documents, ParsedDocument
from src.data_processing.embedder import generate_embeddings_for_chunks, ProcessedChunkWithEmbedding
from src.vector_store.mongodb_manager import MongoDBManager

# Configurer le logging pour le notebook
setup_logging(level="INFO") # Mettre à DEBUG pour plus de détails
logger = logging.getLogger("nb_01_ingestion_embedding")

logger.info("Configuration initiale du notebook terminée.")
logger.info(f"Utilisation de DATA_DIR: {settings.DATA_DIR}")
logger.info(f"PDFs ArXiv seront (télé)chargés depuis/vers: {PDF_INPUT_DIR}")
logger.info(f"Métadonnées ArXiv seront (télé)chargées depuis/vers: {METADATA_INPUT_DIR}")

# --- Affichage de la configuration d'embedding active et vérification des prérequis ---
logger.info(f"--- Configuration d'Embedding Active (depuis settings.py et .env) ---")
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'embedding par défaut configuré : {active_embedding_provider}")

if active_embedding_provider == "openai":
    logger.info(f"  Modèle OpenAI Embedding à utiliser : {settings.OPENAI_EMBEDDING_MODEL_NAME}")
    logger.info(f"  Dimension OpenAI Embedding (configurée) : {settings.OPENAI_EMBEDDING_DIMENSION}")
    if not settings.OPENAI_API_KEY:
        logger.error("ERREUR : Le fournisseur d'embedding est 'openai', mais OPENAI_API_KEY n'est pas configurée dans .env. La génération d'embeddings échouera.")
elif active_embedding_provider == "huggingface":
    logger.info(f"  Modèle HuggingFace Embedding à utiliser : {settings.HUGGINGFACE_EMBEDDING_MODEL_NAME}")
    logger.info(f"  Dimension HuggingFace Embedding (configurée) : {settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION}")
    # Les embeddings HuggingFace locaux (SentenceTransformers) ne nécessitent généralement pas de clé API.
elif active_embedding_provider == "ollama":
    logger.info(f"  Modèle Ollama Embedding à utiliser : {settings.OLLAMA_EMBEDDING_MODEL_NAME}")
    logger.info(f"  Dimension Ollama Embedding (configurée) : {settings.OLLAMA_EMBEDDING_MODEL_DIMENSION}")
    logger.info(f"  URL de base Ollama : {settings.OLLAMA_BASE_URL}")
    if not settings.OLLAMA_BASE_URL:
        logger.error("ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée. La génération d'embeddings échouera.")
    logger.info(f"  ASSUREZ-VOUS que le modèle '{settings.OLLAMA_EMBEDDING_MODEL_NAME}' est disponible sur votre serveur Ollama (ex: via 'ollama pull {settings.OLLAMA_EMBEDDING_MODEL_NAME}').")
else:
    logger.error(f"ERREUR : Fournisseur d'embedding inconnu configuré dans settings.py : '{active_embedding_provider}'. La génération d'embeddings échouera.")
# --- Fin de la section sur la configuration d'embedding ---

# Paramètres pour cette exécution de notebook (pour limiter le nombre d'articles traités)
ARXIV_QUERY_NOTEBOOK = "explainable artificial intelligence for robotics" 
MAX_RESULTS_NOTEBOOK = 2 
COLLECTION_NAME_NOTEBOOK = "arxiv_chunks_notebook_test" 
VECTOR_INDEX_NAME_NOTEBOOK = "vector_index_notebook_test"
TEXT_INDEX_NAME_NOTEBOOK = "text_index_notebook_test"

# Nettoyer les répertoires de données de test (optionnel)
# import shutil
# if PDF_INPUT_DIR.exists():
#     logger.info(f"Nettoyage du répertoire PDF: {PDF_INPUT_DIR}")
#     shutil.rmtree(PDF_INPUT_DIR)
# PDF_INPUT_DIR.mkdir(parents=True, exist_ok=True)
# if METADATA_INPUT_DIR.exists():
#     logger.info(f"Nettoyage du répertoire Metadata: {METADATA_INPUT_DIR}")
#     shutil.rmtree(METADATA_INPUT_DIR)
# METADATA_INPUT_DIR.mkdir(parents=True, exist_ok=True)

### Étape 1: Téléchargement d'Articles depuis ArXiv

Nous utilisons `arxiv_downloader.download_pipeline` pour rechercher et télécharger quelques articles.
Pour ce notebook, nous limitons la recherche à `MAX_RESULTS_NOTEBOOK` articles pour que l'exécution soit rapide.

In [ ]:
logger.info(f"--- Étape 1: Téléchargement d'Articles ArXiv (max: {MAX_RESULTS_NOTEBOOK}) ---")

downloaded_files_info = download_arxiv_papers(
    query=ARXIV_QUERY_NOTEBOOK,
    max_results=MAX_RESULTS_NOTEBOOK 
    # Utilise les sort_by, sort_order, et paths de settings par défaut
)

if downloaded_files_info and downloaded_files_info.get('pdfs'):
    logger.info(f"Téléchargement terminé. {len(downloaded_files_info['pdfs'])} PDFs et {len(downloaded_files_info['metadata'])} fichiers de métadonnées.")
    for pdf_path in downloaded_files_info['pdfs'][:2]: # Afficher les 2 premiers
        logger.info(f"  PDF téléchargé : {pdf_path}")
    for meta_path in downloaded_files_info['metadata'][:2]:
        logger.info(f"  Métadonnées sauvegardées : {meta_path}")
else:
    logger.warning("Aucun fichier PDF n'a été téléchargé. Vérifiez la requête ArXiv ou la connexion.")
    # On pourrait arrêter ici si aucun fichier n'est téléchargé

### Étape 2: Parsing des Documents

Maintenant, nous utilisons `document_parser.parse_document_collection` pour lire les PDFs téléchargés et leurs fichiers de métadonnées JSON associés. Cela extraira le texte brut et structurera les métadonnées.

In [ ]:
logger.info(f"\n--- Étape 2: Parsing des Documents ---")

# parse_document_collection utilise les chemins PDF_INPUT_DIR et METADATA_INPUT_DIR définis dans document_parser.py
# qui sont basés sur settings.DATA_DIR
parsed_documents: List[ParsedDocument] = parse_document_collection()

if parsed_documents:
    logger.info(f"{len(parsed_documents)} documents ont été parsés avec succès.")
    # Afficher un extrait du premier document parsé
    if len(parsed_documents) > 0:
        doc_example = parsed_documents[0]
        logger.info(f"Exemple de document parsé (ID ArXiv: {doc_example['arxiv_id']}):")
        logger.info(f"  Titre (depuis métadonnées): {doc_example['metadata'].get('title', 'N/A')}")
        logger.info(f"  Extrait du texte: '{doc_example['text_content'][:200].replace(chr(10), ' ')}...'")
        logger.info(f"  Chemin PDF: {doc_example['pdf_path']}")
        logger.info(f"  Chemin Métadonnées: {doc_example['metadata_path']}")
else:
    logger.warning("Aucun document n'a été parsé. Vérifiez si des PDFs existent dans le répertoire attendu.")

### Étape 3: Prétraitement du Texte (Nettoyage et Chunking)

Les documents parsés sont maintenant nettoyés et découpés en chunks plus petits et gérables en utilisant `preprocessor.preprocess_parsed_documents`.

In [ ]:
logger.info(f"\n--- Étape 3: Prétraitement du Texte ---")

if parsed_documents:
    processed_chunks = preprocess_parsed_documents(parsed_documents)
    if processed_chunks:
        logger.info(f"{len(processed_chunks)} chunks ont été générés après prétraitement.")
        # Afficher un extrait du premier chunk
        if len(processed_chunks) > 0:
            chunk_example = processed_chunks[0]
            logger.info(f"Exemple de chunk traité (ID: {chunk_example['chunk_id']}):")
            logger.info(f"  ID ArXiv d'origine: {chunk_example['arxiv_id']}")
            logger.info(f"  Titre d'origine: {chunk_example['original_document_title']}")
            logger.info(f"  Extrait du chunk: '{chunk_example['text_chunk'][:200].replace(chr(10), ' ')}...'")
    else:
        logger.warning("Aucun chunk n'a été généré lors du prétraitement.")
else:
    logger.warning("Aucun document parsé à prétraiter. Étape de prétraitement sautée.")
    processed_chunks = [] # S'assurer que la variable existe

### Étape 4: Génération des Embeddings

Chaque chunk de texte est maintenant converti en une représentation vectorielle (embedding) en utilisant `embedder.generate_embeddings_for_chunks`.

In [ ]:
logger.info(f"\n--- Étape 4: Génération des Embeddings ---")
chunks_with_embeddings: List[ProcessedChunkWithEmbedding] = []

if processed_chunks:
    # Vérification proactive des prérequis pour le fournisseur d'embedding configuré
    provider_check_ok = True
    active_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
    if active_provider == "openai" and not settings.OPENAI_API_KEY:
        logger.error("OpenAI est le fournisseur d'embedding, mais OPENAI_API_KEY n'est pas configurée. Impossible de générer les embeddings.")
        provider_check_ok = False
    elif active_provider == "ollama":
        if not settings.OLLAMA_BASE_URL:
            logger.error("Ollama est le fournisseur d'embedding, mais OLLAMA_BASE_URL n'est pas configurée.")
            provider_check_ok = False
        if not settings.OLLAMA_EMBEDDING_MODEL_NAME: # Vérification ajoutée
            logger.error("Ollama est le fournisseur d'embedding, mais OLLAMA_EMBEDDING_MODEL_NAME n'est pas configuré.")
            provider_check_ok = False
    # Pas de vérification de clé pour "huggingface" ici, car les modèles SentenceTransformers locaux n'en nécessitent pas.

    if provider_check_ok:
        logger.info(f"Appel de generate_embeddings_for_chunks avec le provider: {active_provider}")
        chunks_with_embeddings = generate_embeddings_for_chunks(processed_chunks)
        if chunks_with_embeddings:
            logger.info(f"{len(chunks_with_embeddings)} chunks ont maintenant des embeddings.")
            if len(chunks_with_embeddings) > 0:
                chunk_emb_example = chunks_with_embeddings[0]
                logger.info(f"Exemple de chunk avec embedding (ID: {chunk_emb_example['chunk_id']}):")
                logger.info(f"  Fournisseur d'Embedding Utilisé: {chunk_emb_example.get('embedding_provider', 'N/A')}")
                logger.info(f"  Modèle d'Embedding Utilisé: {chunk_emb_example.get('embedding_model', 'N/A')}")
                logger.info(f"  Dimension Réelle de l'Embedding: {chunk_emb_example.get('embedding_dimension', 'N/A')}") 
                logger.info(f"  Vecteur d'Embedding (5 premières dimensions): {chunk_emb_example.get('embedding', [])[:5]}...")
        else:
            logger.warning("Aucun embedding n'a été généré.")
    else:
        logger.warning("Prérequis non remplis pour le fournisseur d'embedding configuré. Étape d'embedding sautée.")
else:
    logger.warning("Aucun chunk traité à embedder. Étape d'embedding sautée.")

### Étape 5: Stockage dans MongoDB et Création des Index

Enfin, les chunks avec leurs embeddings sont insérés dans une collection MongoDB. Nous créons également les index de recherche vectorielle et textuelle nécessaires pour notre moteur RAG.

In [ ]:
logger.info(f"\n--- Étape 5: Stockage MongoDB et Création d'Index ---")
mongo_mgr = None 

if chunks_with_embeddings:
    try:
        logger.info(f"Initialisation de MongoDBManager pour la collection: {COLLECTION_NAME_NOTEBOOK}")
        mongo_mgr = MongoDBManager(mongo_uri=settings.MONGO_URI, db_name=settings.MONGO_DATABASE_NAME)
        mongo_mgr.connect()

        test_collection = mongo_mgr.get_collection(COLLECTION_NAME_NOTEBOOK)
        if test_collection is not None: # Vérifier que la collection a bien été récupérée
            logger.info(f"Suppression des documents existants dans la collection de test '{COLLECTION_NAME_NOTEBOOK}'...")
            delete_result = test_collection.delete_many({})
            logger.info(f"{delete_result.deleted_count} documents supprimés.")
        else:
            logger.error(f"Impossible d'obtenir la collection {COLLECTION_NAME_NOTEBOOK}. L'insertion et la création d'index vont échouer.")
            # Il serait peut-être préférable de lever une exception ici pour arrêter le flux du notebook
            # car les étapes suivantes dépendent de test_collection.
            # Pour l'instant, on logue et on continue, mais les étapes suivantes pourraient échouer.
            # raise ConnectionError(f"Impossible d'obtenir la collection {COLLECTION_NAME_NOTEBOOK}")


        logger.info(f"Insertion de {len(chunks_with_embeddings)} chunks dans MongoDB...")
        insertion_summary = mongo_mgr.insert_chunks_with_embeddings(
            chunks_with_embeddings,
            collection_name=COLLECTION_NAME_NOTEBOOK
        )
        logger.info(f"Résumé de l'insertion MongoDB: {insertion_summary}")

        if insertion_summary.get("inserted_count", 0) > 0:
            logger.info(f"Création/Vérification de l'index vectoriel '{VECTOR_INDEX_NAME_NOTEBOOK}'. La dimension de l'index sera basée sur le fournisseur d'embedding configuré: '{settings.DEFAULT_EMBEDDING_PROVIDER}'.")
            vector_filter_fields = [
                "arxiv_id", 
                "original_document_title", 
                "metadata.primary_category", 
                "embedding_provider", 
                "embedding_model"     
            ]
            success_vector_idx = mongo_mgr.create_vector_search_index(
                collection_name=COLLECTION_NAME_NOTEBOOK,
                index_name=VECTOR_INDEX_NAME_NOTEBOOK,
                embedding_field="embedding",
                filter_fields=vector_filter_fields
            )
            if success_vector_idx:
                logger.info("Index vectoriel géré avec succès.")
            else:
                logger.warning("Problème lors de la gestion de l'index vectoriel.")

            logger.info(f"Création/Vérification de l'index textuel '{TEXT_INDEX_NAME_NOTEBOOK}'...")
            success_text_idx = mongo_mgr.create_text_search_index(
                collection_name=COLLECTION_NAME_NOTEBOOK,
                index_name=TEXT_INDEX_NAME_NOTEBOOK,
                text_field="text_chunk",
                additional_text_fields={"original_document_title": "string", "metadata.title": "string"}
            )
            if success_text_idx:
                logger.info("Index textuel géré avec succès.")
            else:
                logger.warning("Problème lors de la gestion de l'index textuel.")
            
            # S'assurer que test_collection est disponible avant de l'utiliser
            if test_collection is not None:
                # Récupérer un _id valide à partir des chunks insérés
                first_chunk_id = chunks_with_embeddings[0]["chunk_id"]
                sample_doc_from_db = test_collection.find_one({"_id": first_chunk_id})
                if sample_doc_from_db:
                    logger.info(f"Exemple de document récupéré de MongoDB (ID: {sample_doc_from_db['_id']}):")
                    logger.info(f"  Texte: {sample_doc_from_db.get('text_chunk', '')[:100]}...")
                    logger.info(f"  Fournisseur Embedding: {sample_doc_from_db.get('embedding_provider')}")
                    logger.info(f"  Modèle Embedding: {sample_doc_from_db.get('embedding_model')}")
                    logger.info(f"  Dimension Embedding (stockée): {sample_doc_from_db.get('embedding_dimension')}")
                    logger.info(f"  Vecteur Embedding (premières dims): {str(sample_doc_from_db.get('embedding', [])[:3])[:100]}...")
                else:
                    logger.warning(f"Impossible de récupérer le document d'exemple '{first_chunk_id}' depuis MongoDB.")
            else:
                logger.warning("test_collection non disponible, impossible de récupérer un document d'exemple.")
        else:
            logger.warning("Aucun document n'a été inséré, la création des index pourrait ne pas être pertinente ou échouer sur une collection vide.")

    except Exception as e:
        logger.error(f"Erreur lors des opérations MongoDB: {e}", exc_info=True)
    finally:
        if mongo_mgr:
            mongo_mgr.close()
            logger.info("Connexion MongoDB fermée.")
else:
    logger.warning("Aucun chunk avec embedding à stocker. Étape MongoDB sautée.")

logger.info("\nPipeline d'Ingestion et d'Embedding terminé pour ce notebook !")

## Conclusion

Ce notebook a illustré l'ensemble du pipeline d'ingestion :
- Téléchargement des données sources (ArXiv).
- Parsing pour extraire le texte et les métadonnées.
- Prétraitement pour nettoyer et diviser le texte en chunks.
- Génération des embeddings pour chaque chunk.
- Stockage des données enrichies dans MongoDB et création des index nécessaires pour la recherche.

Les données sont maintenant prêtes à être utilisées par le `RetrievalEngine` et les agents du "Cognitive Swarm". Vous pouvez explorer la collection MongoDB (`arxiv_chunks_notebook_test` dans la base `cognitive_swarm_db` par défaut) pour voir les documents stockés.